In [ ]:
!pip install -U rebound

## **CONTOH MASALAH DUA BENDA**

**Akan diperiksa besaran-besaran kekal pada masalah dua benda**, yaitu nilai energi ($E$), momentum sudut ($L$), eksentrisitas ($e$), dan setengah sumbu panjang ($a$). Integrasi akan dicoba menggunakan metode *leapfrog*, sedangkan unit akan merujuk ke setelan baku bawaan (*default*)

**Massa benda kedua akan dicoba dengan dua nilai yang berbeda**: $m_2 = 0$ dan $m_2 = 1 \times 10^{-3}\,M_{\odot}$, sedangkan nilai $a = 1$ au, dan $e = 0,\!3$

Plot orbit akan coba diperlihatkan, sedangkan plot lain yang lebih lanjut bersifat opsional.

In [ ]:
from math import sqrt
import numpy as np
import rebound

In [ ]:
sim = rebound.Simulation()

In [ ]:
# Integrator:
# IAS15 (default), WHFast, SEI, LEAPFROG, JANUS, MERCURIUS,
# WHCKL, WHCKM, WHCKC, SABA4, SABACL4, SABACM4, SABA(10,6,4),
# EOS, BS, WHFast512, none

sim.integrator = 'leapfrog'

In [ ]:
# DEFAULT UNITS (RECOMMENDED for NEWBIE)
sim.units = ('yr', 'au', 'msun')

In [ ]:
# use a fixed time step, in time unit defined above
sim.dt = 1e-4

In [ ]:
# add particles, N = 2. Second body, say, with (in msun) m = 0, or m = 1e-3; a = 1, e = 0.3
sim.add(m = 1.0)
sim.add(m = 1e-3, a = 1.0, e = 0.3)

sim.N_active = sim.N

In [ ]:
# do not forget to move to the center of mass
sim.move_to_com()

In [ ]:
# create time array, e.g. 1 orbit = 2 pi (radian), 250 times 'point' per orbit
Norbits = 1
Nsteps  = Norbits * 250
times   = np.linspace(0, Norbits * 2 * np.pi, Nsteps)

#for i, t in enumerate(times):
#    print(f'{t:.9f}')

In [ ]:
# coordinates for both particles
x = np.zeros((sim.N, Nsteps))
y = np.zeros_like(x)

#print(x, y)

In [ ]:
# energy & angular momentum of the system
energy        = np.zeros(Nsteps)
Lx, Ly, Lz, L = np.zeros(Nsteps), np.zeros(Nsteps), np.zeros(Nsteps), np.zeros(Nsteps)

#print(energy)
#print(Lx, Ly, Lz, L)

In [ ]:
print('---------------------------------------------------------------')
print('     t               E               L             e       a   ')
print('---------------------------------------------------------------')

# now integrate
for i, t in enumerate(times):
    sim.integrate(t, exact_finish_time = 0)

    energy[i] = sim.energy()

    Lx[i], Ly[i], Lz[i] = sim.angular_momentum()
    L[i]                = sqrt(Lx[i]*Lx[i] + Ly[i]*Ly[i] + Lz[i]*Lz[i])

    e = sim.particles[sim.N - 1].e
    a = sim.particles[sim.N - 1].a

    for j in range(sim.N):
        x[j, i] = sim.particles[j].x
        y[j, i] = sim.particles[j].y

    #
    print(f'{t:.9f}\t{energy[i]:.9f}\t{L[i]:.9f}\t{e:.5f}\t{a:.5f}')

print('---------------------------------------------------------------')
print('     t               E               L             e       a   ')
print('---------------------------------------------------------------')

In [ ]:
print(f'Current number of particles        = {sim.N:d}')
print(f'Current number of active particles = {sim.N_active:d}')
print(f'Current number of real particles   = {sim.N_real:d}')
print(f'Integrator                         : {sim.integrator}')
print(f'Simulation units                   : {sim.units}')
print(f'Gravity                            : {sim.gravity}')
print(f'Softening                          = {sim.softening}')
print(f'G constant                         = {sim.G:E}')
print(f'Test particle type                 = {sim.testparticle_type}')
print(f'Time                               = {sim.t:E}')
print(f'Timestep                           = {sim.dt:E}')
print(f'Last timestep done                 = {sim.dt_last_done:E}')
print(f'Python walltime                    = {sim.walltime:E}')
print(f'Simulation status                  : {sim.status}')
print(f'Simulation message                 : {sim.messages}')

In [ ]:
# plot the instantaneous orbits using the built-in REBOUND class OrbitPlot

fig = rebound.OrbitPlot(sim, unitlabel="[au]")

### **PLOT** (Opsional)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# plot the orbit
fig, ax = plt.subplots()
ax.scatter(x, y, s = 2)
ax.set_title('Orbit') # {sim.dt:E}
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.grid(True)
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/rebound/orbit.pdf')
plt.show()

In [ ]:
# plot the energy
fig, ax = plt.subplots()
ax.scatter(times, np.abs(energy - energy[0]) / np.abs(energy[0]), s = 2)
#print(energy)
ax.set_title('Energy') # sim.dt
ax.set_xlabel('time')
ax.set_yscale('log')
ax.set_ylabel('energy')
plt.grid(True)
plt.savefig('/content/gdrive/My Drive/Colab Notebooks/rebound/energy.pdf')
plt.show()

In [ ]:
print('Done')

In [ ]:
# to free the memory
sim = None